In [1]:
from snorkel.labeling import labeling_function
import json
import os
import numpy as np
from snorkel.labeling import LFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from transformers import CLIPProcessor, CLIPModel

In [2]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

@labeling_function()
def llava_7b(image_name):
    root_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/total_results/'
    llava_7b_results = 'llava:7b_results_hateful.json'
    path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(path_to_llava_7b_results, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1

@labeling_function()
def llava_13b(image_name):
    root_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/total_results/'
    llava_7b_results = 'llava 13b-allsamples-results.json'
    path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(path_to_llava_7b_results, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1

@labeling_function()
def bakllava(image_name):
    root_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/total_results/'
    llava_7b_results = 'bakllava-allsamples-results.json'
    path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(path_to_llava_7b_results, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1

@labeling_function()
def llava_llama3(image_name):
    root_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/total_results/'
    llava_7b_results = 'llava-llama3-allsamples-results.json'
    path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(path_to_llava_7b_results, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1

@labeling_function()
def llava_phi3(image_name):
    root_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/total_results/'
    llava_7b_results = 'llava-phi3-allsamples-results.json'
    path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(path_to_llava_7b_results, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1


@labeling_function()
def moondream(image_name):
    root_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/total_results/'
    llava_7b_results = 'moondream-allsamples-results.json'
    path_to_llava_7b_results = os.path.join(root_path,llava_7b_results)
    with open(path_to_llava_7b_results, 'r') as file:
        data = json.load(file)

    return data[image_name] if data[image_name] is not None else -1

In [3]:
train_data_json_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/simplified_train.json'
dev_data_json_path = '/home/macula/SMATousi/CVPR/ViGIR_CVPR_LLM/prompting_framework/prompting_results/hateful/simplified_dev.json'

with open(train_data_json_path, 'r') as file:
    train_data = json.load(file)

# Extract and pad image names, ensuring they are 5 digits long before the '.png'
train_image_names = []
for entry in train_data:
    img_name, ext = entry['img'].split('.')
    padded_img_name = img_name.zfill(5)  # Pad the image name to 5 digits
    train_image_names.append(f"{padded_img_name}.{ext}")

with open(dev_data_json_path, 'r') as file:
    dev_data = json.load(file)
    
dev_image_names = []
Y_dev = []
for entry in dev_data:
    Y_dev.append(entry['label'])
    img_name, ext = entry['img'].split('.')
    padded_img_name = img_name.zfill(5)  # Pad the image name to 5 digits
    dev_image_names.append(f"{padded_img_name}.{ext}")

print(f"There are {len(train_image_names)} images in the Train set.")
print(f"There are {len(dev_image_names)} images in the dev set.")
print(f"There are {len(Y_dev)} labels in the dev set.")


There are 8500 images in the Train set.
There are 500 images in the dev set.
There are 500 labels in the dev set.


In [4]:

lfs = [llava_7b,
       moondream,
       llava_llama3,
       llava_phi3,
       bakllava
       ]

applier = LFApplier(lfs)

In [5]:
L_dev = applier.apply(dev_image_names)
L_train = applier.apply(train_image_names)

500it [00:04, 101.48it/s]
8500it [00:44, 190.24it/s]


In [6]:
Y_dev = np.array(Y_dev)
LFAnalysis(L_dev, lfs).lf_summary(Y_dev)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
llava_7b,0,"[0, 1]",1.00,1.00,0.716,298,202,0.596000
moondream,1,"[0, 1]",1.00,1.00,0.716,246,254,0.492000
llava_llama3,2,"[0, 1]",1.00,1.00,0.716,278,222,0.556000
llava_phi3,3,"[0, 1]",0.99,0.99,0.708,267,228,0.539394
bakllava,4,"[0, 1]",0.95,0.95,0.674,276,199,0.581053


In [7]:
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, Y_dev, n_epochs=5000, log_freq=500, seed=12345)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|███████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:01<00:00, 3063.72epoch/s]
INFO:root:Finished Training


In [8]:
probs_dev = label_model.predict_proba(L_dev)
preds_dev = probs_to_preds(probs_dev)
print(
    f"Label model f1 score: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='f1')}"
)
print(
    f"Label model roc-auc: {metric_score(Y_dev, preds_dev, probs=probs_dev, metric='roc_auc')}"
)

Label model f1 score: 0.5470085470085471
Label model roc-auc: 0.5982000000000001


In [9]:
class HatefulMemesDataset(Dataset):
    def __init__(self, image_names, root_dir, labels, processor):
        """
        Args:
            data_frame (DataFrame): DataFrame containing image names and labels.
            image_dir (str): Directory where the images are stored.
            processor (CLIPProcessor): CLIP processor for preprocessing images.
        """
        self.image_names = image_names
        self.root_dir = root_dir
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        # Get image name and label from the dataframe
        img_name = os.path.join(self.root_dir, self.image_names[idx])
        label = self.labels[idx]

        # Load and process image
        image = Image.open(img_name).convert('RGB')
        inputs = self.processor(images=image, return_tensors="pt")

        # Return image and label
        return inputs['pixel_values'].squeeze(0), torch.tensor(label, dtype=torch.long)

# MLP head to be added after the CLIP model
class MLPHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLPHead, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.fc(x)

# CLIP model with MLP head for binary classification
class CLIPWithMLP(nn.Module):
    def __init__(self, clip_model, mlp_head):
        super(CLIPWithMLP, self).__init__()
        self.clip_model = clip_model
        self.mlp_head = mlp_head

        # Freeze CLIP's parameters
        for param in self.clip_model.parameters():
            param.requires_grad = False

    def forward(self, image):
        # Extract image features from CLIP
        image_features = self.clip_model.get_image_features(pixel_values=image)
        # Pass through the MLP head
        outputs = self.mlp_head(image_features)
        return outputs

# Training function
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Evaluation function to compute precision, recall, and F1-score
def evaluate_model(model, dev_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in tqdm(dev_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Compute metrics
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return precision, recall, f1

In [12]:
root_dir = "/home1/pupil/goowfd/CVPR_2025/hateful_memes/img/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

probs_train = label_model.predict_proba(L_train)
preds_train = probs_to_preds(probs_train)

# Create datasets and dataloaders
train_dataset = HatefulMemesDataset(image_names=train_image_names, root_dir=root_dir, labels=preds_train, processor=processor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=16)

dev_dataset = HatefulMemesDataset(image_names=dev_image_names, root_dir=root_dir, labels=Y_dev, processor=processor)
dev_loader = DataLoader(dev_dataset, batch_size=8, shuffle=False)

# Define MLP head (the dimension is based on CLIP output size)
mlp_head = MLPHead(input_dim=512, output_dim=2)  # Binary classification, so output_dim = 2

# Create the full model with CLIP + MLP
model = CLIPWithMLP(clip_model=clip_model, mlp_head=mlp_head)
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.mlp_head.parameters(), lr=0.0001)

# Train the model
epochs = 100
train_model(model, train_loader, criterion, optimizer, device, epochs=epochs)

# Evaluate the model
evaluate_model(model, dev_loader, device)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:08<00:00,  7.57it/s]


Epoch [1/100], Loss: 0.6603


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:08<00:00,  7.65it/s]


Epoch [2/100], Loss: 0.5679


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:08<00:00,  7.60it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:09<00:00,  7.21it/s]


Epoch [100/100], Loss: 0.0246


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:05<00:00, 11.65it/s]

Precision: 0.5751
Recall: 0.5360
F1 Score: 0.5549


(0.575107296137339, 0.536, 0.5548654244306418)